In [42]:
#import tensorflow as tf
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import chi2
from math import floor
from sklearn.svm import LinearSVC
import operator
from functools import reduce
from Utilities.G2Dependency import *
from Utilities.ClassicMM import *
from ipynb.fs.defs.GraphReader import GraphReader
from random import shuffle

In [43]:
class MMPC_opt():
    def __init__(self, alpha):
        
        self._alpha = alpha
        self._G2Dep = None
        self._arrX= None
        return
        
    def fit(self, X):
        """
        Fit training dataset X, initializing all the G2Dep matrix
        """
        x_heads = []
        for i in range(0,len(X.keys())):
            max_v=1
            for row in X[X.keys()[i]]:
                if (row>max_v-1):
                    max_v+=row-(max_v-1)
            x_heads.append("Node"+str(i+1)+"@"+str(max_v))
        X.columns = x_heads
        
        self._arrX = arrayX(X)
        self._G2Dep = G2Dependency(self._alpha, len(self._arrX[0]['data']))
        
        
        for i in range(0,len(self._arrX)):
            for j in range(0,len(self._arrX)):
                if (i != j):
                    self._G2Dep.dependency(self._arrX[i], self._arrX[j], [], self._alpha)
        print("Finished fitting of X in MMPC object")
        return

        
    def evaluate(self, index):
        
        start = time.clock()
        CPC=[]
        alpha = self._alpha
        X = self._arrX
        TargetNode = [x for x in self._arrX if x['name'].split('@')[0] == ("Node"+str(index+1))][0]
        Universe = arrayUniverse(TargetNode['name'], self._arrX)
        
        
        CPC=[]
        print("Entering Phase I")
        print("MMPC_beggining: \n"+str(len(Universe)))
        while len(Universe)>0:
            CPC_old = list(CPC) # copy
            maxminheur=self.__mm_heuristic(TargetNode,CPC,list(Universe),X,alpha)
            F = maxminheur[0]
            assocF = maxminheur[1]
            Universe = maxminheur[2]
            if assocF > 0:
                CPC.append(F)
                indF=Universe.index(F)
                Universe.pop(indF)
            #if (len(CPC)==len(CPC_old)) or (len(CPC)>0.3*(len(Universe)-1)):
            if (len(CPC)==len(CPC_old)):
                break
            print("\nUniverse actual size:")
            print(len(Universe))
            print("CPC actual size:")
            print(len(CPC))
            print("CPC contents:")
            print([x['name'] for x in CPC])

        # Phase 2: Backward
        print("\nEntering Phase II")
        CPC=CPC[::-1]
        if len(CPC)>1:
            Z=list(CPC)
            for i in range(len(CPC)-1,-1,-1):
                # index is i
                Z.pop(i)
                #print("Analyzing D-separator for ",CPC[i]['name'])
                if self.__exist_dseparator(TargetNode,CPC[i],Z,X,alpha)[0] == True:
                    #print("it did exist! removing from cpc.")
                    CPC.pop(i)
        stop = time.clock()
        results = {}
        results['CPC'] = CPC
        results['time'] = stop - start
        return results
        
    def __exist_dseparator(self, TargetNode,Xi, Z, X, alpha):
        flagExist = False
        dsepSet=[]
        #counter=0
        #print_names(Z)
        for i in range(0,(2**len(Z))-1):
            IDsubsetZ_dec = i
            IDsubsetZ_bin = bin(IDsubsetZ_dec)
            subsetZ = getZsubset(IDsubsetZ_bin,Z)
            # no cache
            #print("from exist dseparator")
            dep = self._G2Dep.dependency(TargetNode,Xi,subsetZ, alpha)
            #print(subsetZ)
            #print(dep)
            if (dep==0):
                flagExist = True
                dsepSet = subsetZ
                break
        #print("Module exist d-separator: ",counter)
        return [flagExist,dsepSet]

In [44]:
class MMPC_opt(MMPC_opt):
    def __mm_heuristic(self, TargetNode, CPC, Universe, X, alpha):
        F=[]
        assocF=-1
        Z = CPC
        fixedCondVars = []
        if (len(CPC)>0):
            Z = CPC[0:-1]           # all but the last one   
            fixedCondVars = [CPC[-1]] # we use last one
        for i in range(len(Universe)-1,-1,-1):
            if (len(Universe[i])==0):
                continue
            assoc = self.__min_assoc(TargetNode,Universe[i],Z,fixedCondVars,X,alpha)
            if (assoc>assocF):
                assocF = assoc
                F = Universe[i]
            if (assoc==0):
                Universe.pop(i)
        return [F,assocF,Universe]
    
    
    def __min_assoc(self, TargetNode, Xi,Z, fixedCondVars, X, alpha):
        min_assoc=999
        if len(Z)==0:
            min_assoc = self._G2Dep.dependency(TargetNode, Xi, fixedCondVars, alpha)
            subsetZ_min_assoc = fixedCondVars        
        else:
            #print(2**len(Z)-1)
            for IDsubsetZ_dec in range(0,2**len(Z)-1):
                IDsubsetZ_bin = bin(IDsubsetZ_dec)
                subsetZ = getZsubset(IDsubsetZ_bin,Z)            
                subsetZ_assoc=self._G2Dep.dependency(TargetNode, Xi, fixedCondVars+subsetZ,alpha)
                #counter+=1
                #print(subsetZ_assoc[IDsubsetZ_dec])
                if subsetZ_assoc < min_assoc:
                    min_assoc = subsetZ_assoc
                    subsetZ_min_assoc = fixedCondVars + subsetZ
                    if (min_assoc==0):
                        break
        return min_assoc
    
    def shuffle_rows(self):
        shuffle(self._arrX)
        
        return

In [62]:
MMPC_model = MMPC_opt(0.05)

In [63]:
X = pd.read_csv("/home/a20114261/sdelrio/alarm_datasets/Alarm10/Alarm10_s5000_v8.txt",delimiter='  ',header=None, engine='python')

In [64]:
MMPC_model.fit(X)

Succesful instantiation of Dep object
Finished fitting of X in MMPC object


In [48]:
MMPC_model.shuffle_rows()

In [66]:
%%time
cpc = MMPC_model.evaluate(4)
print("Results")
#print(len([x['name'] for x in cpc]))
print([x['name'] for x in cpc['CPC']])

Entering Phase I
MMPC_beggining: 
369

Universe actual size:
36
CPC actual size:
1
CPC contents:
['Node83@2']

Universe actual size:
33
CPC actual size:
2
CPC contents:
['Node83@2', 'Node74@2']

Universe actual size:
25
CPC actual size:
3
CPC contents:
['Node83@2', 'Node74@2', 'Node43@2']

Universe actual size:
24
CPC actual size:
4
CPC contents:
['Node83@2', 'Node74@2', 'Node43@2', 'Node31@2']

Universe actual size:
22
CPC actual size:
5
CPC contents:
['Node83@2', 'Node74@2', 'Node43@2', 'Node31@2', 'Node28@2']

Universe actual size:
21
CPC actual size:
6
CPC contents:
['Node83@2', 'Node74@2', 'Node43@2', 'Node31@2', 'Node28@2', 'Node22@4']

Universe actual size:
20
CPC actual size:
7
CPC contents:
['Node83@2', 'Node74@2', 'Node43@2', 'Node31@2', 'Node28@2', 'Node22@4', 'Node29@3']

Universe actual size:
19
CPC actual size:
8
CPC contents:
['Node83@2', 'Node74@2', 'Node43@2', 'Node31@2', 'Node28@2', 'Node22@4', 'Node29@3', 'Node21@4']

Universe actual size:
18
CPC actual size:
9
CPC c

KeyboardInterrupt: 

In [57]:
# print([x['name'] for x in cpc])

['Node18@3', 'Node326@2', 'Node21@4', 'Node228@2', 'Node19@3', 'Node22@4', 'Node15@3', 'Node4@4', 'Node1@4', 'Node23@4', 'Node25@4', 'Node27@2', 'Node29@3']
